# Technical Factor - MIKE Index (MIKE)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        close,
        (high + low + close) / 3 AS TYP,
        TYP + (TYP - m_min(low, 12)) AS WR,
        TYP - (m_max(high, 12) - TYP) AS WS,
        TYP + (m_max(high, 12) - m_min(low, 12)) AS MR,
        TYP - (m_max(high, 12) - m_min(low, 12)) AS MS,
        m_max(high, 12) + 2 * (TYP - m_min(low, 12)) AS SR,
        m_min(low, 12) - 2 * (m_max(high, 12) - TYP) AS SS,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(close > TYP AND m_lag(close, 1) <= m_lag(TYP, 1), 1, 0) AS TRBY1,
        IF(close < TYP AND m_lag(close, 1) >= m_lag(TYP, 1), 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(close > WR AND (close - m_lag(close, 1)) > 0, 1, 0) AS MTBY1,
        IF(close < WS AND (close - m_lag(close, 1)) < 0, 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(close < SS AND close > m_lag(close, 1), 1, 0) AS RVBY1,
        IF(close > SR AND close < m_lag(close, 1), 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(close, 1) <= m_lag(MR, 1) AND close > MR, 1, 0) AS BKBY1,
        IF(m_lag(close, 1) >= m_lag(MS, 1) AND close < MS, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        close,
        TYP,
        WR,
        WS,
        MR,
        MS,
        SR,
        SS,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend    USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,close,TYP,WR,WS,MR,MS,SR,SS,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-01-17,600519.SH,11850.263406,11844.506962,12105.447631,11270.654716,12679.299878,11009.714047,12940.240547,10435.861800,1,0,0,0,0,0,0,0
1,2025-01-20,600519.SH,12013.588913,12014.104821,12444.643347,11850.154794,12608.593374,11419.616267,13039.131901,11255.666240,0,1,0,0,0,0,0,0
2,2025-01-21,600519.SH,11959.418608,11988.472355,12393.378417,11841.248598,12540.602174,11436.342537,12945.508235,11289.118780,0,0,0,0,0,0,0,0
3,2025-01-22,600519.SH,11738.257136,11803.288655,12023.011016,11470.881198,12355.418474,11251.158836,12575.140835,10918.751379,0,0,0,0,0,0,0,0
4,2025-01-23,600519.SH,11754.548957,11808.855027,12034.143761,11482.013942,12360.984846,11256.725209,12586.273579,10929.884123,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,2025-12-25,600519.SH,11945.482630,11924.421463,12360.759202,11858.506769,12426.673896,11422.169030,12863.011635,11356.254336,1,0,0,0,0,0,0,0
228,2025-12-26,600519.SH,11945.144750,11947.622535,12254.588218,11904.908912,12297.301840,11597.943229,12604.267524,11555.229606,0,1,0,0,0,0,0,0
229,2025-12-29,600519.SH,11842.682738,11874.021078,12007.552480,11757.705998,12123.867559,11624.174597,12257.398961,11507.859517,0,0,0,0,0,0,0,0
230,2025-12-30,600519.SH,11738.953676,11762.774194,11818.017521,11535.212230,12045.579485,11479.968903,12100.822812,11252.406939,0,0,0,0,0,0,0,0
